In [146]:
#!pip install -q -U kaggle_environments

In [147]:
#pip install termcolor

In [148]:
#pip install pygame

In [149]:
#pip install chart-studio

In [150]:
#pip install cufflinks

### Подргужаем необходимые библиотеки по "Python фен-шуй", то есть сначала import, далее from

In [152]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks

from kaggle_environments import make, evaluate
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)
%matplotlib inline

### С помощью "магии" %%writefile  будем записывать всю информацию в соответствующие файлы, которая генерируется в ячейке.  

 ## Воспользуемся функцией evaluate из библиотеки kaggle_environments с помощью которой запустим наших агентов и проведем эксперимент на заданном количестве игр

## 1. Опишем поведение агента, всегда играющего "камень" - это значение 0

In [156]:
%%writefile rock_agent.py

#0 - rock
#1 - paper
#2 - scissors
def rock_agent(observation, configuration):
    return 0

Overwriting rock_agent.py


In [157]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[-98.0, 98.0]]

## 2. Опишем поведение агента, всегда играющего "бумага" - это значение 1

In [159]:
%%writefile paper_agent.py

def paper_agent(observation, configuration):
    return 1

Overwriting paper_agent.py


In [160]:
evaluate(
    "rps", #environment to use - no need to change
    ["paper_agent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[-97.0, 97.0]]

## 3. Опишем поведение агента, всегда играющего "ножницы" - это значение 2

In [162]:
%%writefile scissors_agent.py

def scissors_agent(observation, configuration):
    return 2

Overwriting scissors_agent.py


In [163]:
evaluate(
    "rps", #environment to use - no need to change
    ["scissors_agent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[-99.0, 99.0]]

## 4. Опишем поведение агента, всегда играющего "рандомно"

In [165]:
%%writefile random_opponent.py

import random

def random_opponent(observation, configuration):
    return random.randrange(0, configuration.signs)

Overwriting random_opponent.py


In [166]:
evaluate(
    "rps", #environment to use - no need to change
    ["random_opponent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[0, 0]]

## 5. Попробуем теперь использовать информацию о прошлых действиях противника. Опишем агента, который производит то же самое действие, что и оппонент на прошлом ходу

In [168]:
%%writefile copy_opponent.py

import random

def copy_opponent(observation, configuration):
    if observation.step > 0:
        return observation.lastOpponentAction
    else:
        return random.randrange(0, configuration.signs)

Overwriting copy_opponent.py


In [169]:
evaluate(
    "rps", #environment to use - no need to change
    ["copy_opponent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[-20.0, 20.0]]

## 6. Опишем поведение агента, который производит действие+1 к тому, что использовал оппонент на прошлом ходу

In [171]:
%%writefile copynext_opponent.py

import random

def copynext_opponent(observation, configuration):
    if observation.step > 0:
        return (observation.lastOpponentAction + 1) % configuration.signs
    else:
        return random.randrange(0, configuration.signs)

Overwriting copynext_opponent.py


In [172]:
evaluate(
    "rps", #environment to use - no need to change
    ["copynext_opponent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[49.0, -49.0]]

## 7. Опишем поведение агента, который производит действие-1 к тому, что использовал оппонент на прошлом ходу

In [174]:
%%writefile copyprevious_opponent.py

import random

def copyprevious_opponent(observation, configuration):
    if observation.step > 0:
        return (observation.lastOpponentAction + 2) % configuration.signs
    else:
        return random.randrange(0, configuration.signs)

Overwriting copyprevious_opponent.py


In [175]:
evaluate(
    "rps", #environment to use - no need to change
    ["copyprevious_opponent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[-99.0, 99.0]]

## 8. Опишем поведение агента, который равновероятно играет таким образом, что не выкинул противник на прошлом ходу

In [177]:
%%writefile opposit_agent.py

import random

def opposit_agent(observation, configuration):
    if observation.step > 0:
        if observation.lastOpponentAction == 0:
            return random.choice([1,2])
        elif observation.lastOpponentAction == 1:
            return random.choice([0,2])
        else:
            return random.choice([1,0])
    else:
        return random.randrange(0, configuration.signs)

Overwriting opposit_agent.py


In [178]:
evaluate(
    "rps", #environment to use - no need to change
    ["opposit_agent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[0, 0]]

## 9. Опишем поведение когда агент выбрасывает три раза одно и то же, после чего меняет выбор на следующее случайное решение

In [180]:
%%writefile random3_opponent.py

import random

counter = 0
stable_sign = None

def random3_opponent(observation, configuration):
    global counter
    global stable_sign
    if observation.step == 0:
        stable_sign = random.randrange(0, configuration.signs)
    elif counter % 3 == 2:
        stable_sign = random.randrange(0, configuration.signs)
    counter += 1
    return stable_sign

Overwriting random3_opponent.py


In [181]:
evaluate(
    "rps", #environment to use - no need to change
    ["random3_opponent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[0, 0]]

## 10 .Опишем поведение, когда агент производит то же действие, если выиграл, и меняет на +1, если не выиграл

In [183]:
%%writefile win_reaction1.py

import math

def get_score(left_move, right_move):
    delta = (
        right_move - left_move
        if (left_move + right_move) % 2 == 0
        else left_move - right_move
    )
    return 0 if delta == 0 else math.copysign(1, delta)

import random

lastAction = None

def win_reaction1(observation, configuration):
    global lastAction
    if observation.step == 0:
        lastAction = random.randrange(0, configuration.signs)
    elif get_score(lastAction, observation.lastOpponentAction) < 1:
        lastAction = (lastAction + 1) % configuration.signs
    return lastAction

Overwriting win_reaction1.py


In [184]:
evaluate(
    "rps", #environment to use - no need to change
    ["win_reaction1.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[27.0, -27.0]]

## 11. Опишем поведение, когда агент производит то же действие, если выиграл, и меняет на -1, если не выиграл

In [186]:
%%writefile win_reaction2.py

import math

def get_score(left_move, right_move):
    delta = (
        right_move - left_move
        if (left_move + right_move) % 2 == 0
        else left_move - right_move
    )
    return 0 if delta == 0 else math.copysign(1, delta)

import random

lastAction = None

def win_reaction2(observation, configuration):
    global lastAction
    if observation.step == 0:
        lastAction = random.randrange(0, configuration.signs)
    elif get_score(lastAction, observation.lastOpponentAction) < 1:
        lastAction = (lastAction + 2) % configuration.signs
    return lastAction

Overwriting win_reaction2.py


In [187]:
evaluate(
    "rps", #environment to use - no need to change
    ["win_reaction2.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[57.0, -57.0]]

## 12. Опишем поведение агента, который  производит то же действие, если выиграл, и меняет на рандомное, если проиграл

In [189]:
%%writefile win_opposit1.py

import math

def get_score(left_move, right_move):
    delta = (
        right_move - left_move
        if (left_move + right_move) % 2 == 0
        else left_move - right_move
    )
    return 0 if delta == 0 else math.copysign(1, delta)

import random

lastAction = None

def win_opposit1(observation, configuration):
    global lastAction
    if observation.step == 0:
        lastAction = random.randrange(0, configuration.signs)
    elif get_score(lastAction, observation.lastOpponentAction) < 1:
        lastAction = observation.lastOpponentAction
    return lastAction

Overwriting win_opposit1.py


In [190]:
evaluate(
    "rps", #environment to use - no need to change
    ["win_opposit1.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[-21.0, 21.0]]

## 13. Опишем поведение, когда агент производит то же действие, если проиграл, и меняет на действие оппонента, если выиграл

In [192]:
%%writefile win_opposit2.py

import math

def get_score(left_move, right_move):
    delta = (
        right_move - left_move
        if (left_move + right_move) % 2 == 0
        else left_move - right_move
    )
    return 0 if delta == 0 else math.copysign(1, delta)

import random

lastAction = None

def win_opposit2(observation, configuration):
    global lastAction
    if observation.step == 0:
        lastAction = random.randrange(0, configuration.signs)
    elif get_score(lastAction, observation.lastOpponentAction) == 1:
        lastAction = observation.lastOpponentAction
    return lastAction

Overwriting win_opposit2.py


In [193]:
evaluate(
    "rps", #environment to use - no need to change
    ["win_opposit2.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[-99.0, 99.0]]

### 14. Опишем поведение, когда агент играет по очереди камень, бумагу, ножницы

In [195]:
%%writefile sequence_agent.py

counter = -1

def sequence(observation, configuration):
    global counter
    counter += 1
    if counter % 3 == 0:
        return 0
    elif counter % 3 == 1:
        return 1
    else:
        return 2

Overwriting sequence_agent.py


In [196]:
evaluate(
    "rps", #environment to use - no need to change
    ["sequence_agent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[0, 0]]

## **Проведём соревнование среди агентов ⏰ В переменной times устанавливаем количество партий (3)**


In [198]:
agents = ['rock_agent.py',
          'paper_agent.py',
          'scissors_agent.py',
          'random_opponent.py',
          'copy_opponent.py',
          'copynext_opponent.py',
          'copyprevious_opponent.py',
          'opposit_agent.py', 
          'random3_opponent.py', 
          'win_reaction1.py', 
          'win_reaction2.py', 
          'win_opposit1.py', 
          'win_opposit2.py', 
           'sequence_agent.py']


rank = {agent: 0 for agent in agents}

for times in range(3):
# число раундов для каждой пары
          
    for i in range(len(agents)):
        for j in range(i+1, len(agents)):
            e = evaluate(
            "rps", #environment to use - no need to change
            [agents[i], agents[j]], #agents to evaluate
            configuration={"episodeSteps": 100, 'tieRewardThreshold': 1} #number of episodes 
            )
            if e[0][0] > e[0][1]:
                rank[agents[i]] += 1
            elif e[0][0] < e[0][1]:
                rank[agents[j]] += 1
            


## **Выведем получившийся результат в виде таблицы Pandas**

In [200]:
result = pd.Series(rank, name="Рейтинг турнира по количеству выигранных партий").sort_values(ascending=False).to_frame()
result

,Рейтинг турнира по количеству выигранных партий
copynext_opponent.py,31
win_reaction1.py,26
opposit_agent.py,23
win_reaction2.py,23
random_opponent.py,16
copy_opponent.py,16
scissors_agent.py,15
random3_opponent.py,15
rock_agent.py,13
paper_agent.py,13


## Представим сведения в виде графика

In [202]:
result.iplot(kind='bar', xTitle='Rank of agents',
                  yTitle='count', title='Distribution')